In [1]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}

def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")

        #for token in jersey_tokens:
        #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

def in_season_around(player_id, curr_season, age):
    if age == "Not Listed": age = 30
    age = int(age)
    year_before = int(curr_season) -1
    year_after = int(curr_season) + 1
    before_season = transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_before)]
    after_season =  transfermarkt_data[(transfermarkt_data['PlayerID'] == player_id) & (transfermarkt_data['Season'] == year_after)]
    if before_season.empty and after_season.empty:
        return -1, "Both Empty"
    if not before_season.empty and after_season.empty:
        if before_season.iloc[0]['Market Value'] != "-":
            if before_season.iloc[0]['Market Value'] == 0:
                return 0, "Only Before + Zero Before"
            else:
                if age >= 30:
                    return (float(before_season.iloc[0]['Market Value']) * .8), "Only Before"
                else:
                    return (float(before_season.iloc[0]['Market Value']) * 1.2), "Only Before"
        else:
            return 0, "Only Before X"
    if not after_season.empty and before_season.empty:
        
        if after_season.iloc[0]['Market Value'] != "-":
            if after_season.iloc[0]['Market Value'] == 0:
                return 0, "Only After + Zero After"
            else:
                if age <= 30:
                    return (float(after_season.iloc[0]['Market Value']) * .8), "Only After"
                else:
                    return (float(after_season.iloc[0]['Market Value']) * 1.2), "Only After"
        else:
            return 0, "Only After X"
    if not before_season.empty and not after_season.empty:
        before_season_val = before_season.iloc[0]['Market Value']
        after_season_val = after_season.iloc[0]['Market Value']
        if before_season_val != "-" and after_season_val != "-":
            return ((float(before_season.iloc[0]['Market Value']) + float(after_season.iloc[0]['Market Value'])) / 2), "Used Average of Season Before and After"
        elif before_season_val != "-" and after_season_val == "-":
            if age >= 30:
                return (float(before_season.iloc[0]['Market Value']) * .8), "Used Season Before * .8 But Had Both"
            else:
                return (float(before_season.iloc[0]['Market Value']) * 1.2), "Used Season Before * 1.2 But Had Both"
        elif before_season_val == "-" and after_season_val != "-":
            if age <= 30:
                return (float(after_season.iloc[0]['Market Value']) * .8), "Used Season After * .8 But Had Both"
            else:
                return (float(after_season.iloc[0]['Market Value']) * 1.2), "Used Season After * 1.2 But Had Both"
        else:
            return 0, "Had Both but Both Were Empty"

def find_in_transfermarkt(player_name, season, country_code, transfermarkt_data):
    row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == season) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_up_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)+1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    one_down_row = transfermarkt_data[(transfermarkt_data['Name'] == player_name) & (transfermarkt_data['Season'] == int(season)-1) & (transfermarkt_data['Team 1 Code'] == country_code)]
    #print(len(row), len(one_down_row), len(one_up_row))
    if not row.empty:
        player_id = row.iloc[0]['PlayerID']
        age = row.iloc[0]['Age']
        marketval = row.iloc[0]['Market Value']
        if marketval != 0:
            return marketval, "Found in Curr Season"
        else:
            new_mv, reason = in_season_around(player_id, season, age)
            print(player_name, new_mv, reason)
            if new_mv == -1 or new_mv == 0 or new_mv == "-":
                return 0, reason
            else:
                return new_mv, reason
    elif not one_up_row.empty:
        player_id = one_up_row.iloc[0]['PlayerID']
        age = one_up_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    elif not one_down_row.empty:
        player_id = one_down_row.iloc[0]['PlayerID']
        age = one_down_row.iloc[0]['Age']
        new_mv, reason = in_season_around(player_id, season, age)
        if new_mv == -1 or new_mv == 0 or new_mv == "-":
            return 0, reason
        else:
            return new_mv, reason
    else:
        return 0, "Did Not Find in Curr Season, One Up, or One Down"

def last_name_match(name, dataset_of_names):
    matching_names = []
    for full_name in dataset_of_names:
        
        # Split the full name into tokens
        full_name_tokens = set(re.split(r'\s+|[-–]', full_name))
        
        # Check if any token matches the last name or its unidecoded version
        for token in full_name_tokens:
            #if full_name == 'Fródi Benjaminsen':
                #print(token, name, token == name)
                #print(full_name_tokens)
            
            if token == name or unidecode(token) == name:
                matching_names.append(full_name)
                break  # Once a match is found, move to the next name
    return matching_names

def check_other_team(lookup_row, transfermarkt_data, results_df):
    new_rows = []
    wrong_rows = []

    name = lookup_row['Name']
    original_jersey = lookup_row['ORIGINAL JERSEY']
    season = lookup_row['Season']
    original_country_code = lookup_row['Team Country Code']
    match_id = lookup_row['Match ID']
    opposing_country_code = ""
    game_row = results_df[results_df['Match ID'] == match_id]
    
    if len(game_row) != 1:
        raise ValueError("There should only be one match per ID")

    team2_code = game_row.iloc[0]['Team 2 Code']
    team1_code = game_row.iloc[0]['Team 1 Code']

    if original_country_code == team1_code:
        opposing_country_code = team2_code
    elif original_country_code == team2_code:
        opposing_country_code = team1_code
    else:
        raise ValueError("Original country code does not match any team in the match")

    dataset_nationality = transfermarkt_data[transfermarkt_data['Team 1 Code'] == opposing_country_code]['Name'].unique()
    possible_names = threshold_player_match(original_jersey, dataset_nationality)
    
    return possible_names[0]

def add_correct_money(money_column):

    if 'm' in money_column:
        value_rough = money_column.split('m')[0].split('€')[1]
        millinum = (float(value_rough) * 1000000)
        return millinum
    elif 'k' in money_column:
        #print(i, 'k')
        value_rough = money_column.split('k')[0].split('€')[1]
        millinum = (float(value_rough) * 1000)

        return millinum
    elif '-' in money_column:
        return 0
    else:
        return 0
  

def remove_nation(match, nation1):
    # Split the match string by 'vs' and strip any leading or trailing spaces
    parts = [part.strip() for part in match.split('vs')]
    # Remove the nation1 from the list of parts
    parts.remove(nation1)
    # Join the remaining parts and return
    return ''.join(parts)

/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [2]:
big_df_combined = load_csv_dataset('combined_DF_merged_April_24.csv')

#PINK GROUP
pink_group_cyrillic = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found (light Pink in XL)/Cyrillic_Guys_Nobody_Found.csv')
pink_group_odd_name_cases = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found (light Pink in XL)/odd_name_found_0_cases.csv')

pink_group_onlone_lookup = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found (light Pink in XL)/Online lookup required - 0 names found.csv')


#RED GROUP

red_group = load_csv_dataset('CSVs of edge cases - for F/IN RED IN EXCEL. DID NOT WORK./Wrong DFs Merged - Complete.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_59250/3762744665.py:49: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [3]:
big_df_combined['Competition'].value_counts()

Competition
World Cup Qualifiers, Europe, 2018           6067
World Cup Qualifiers, Europe, 2022           5628
Euros 2020, Qualifiers                       5568
Euros 2024, Qualifiers                       4907
World Cup Qualifiers, Asia, 2022             4632
World Cup Qualifiers, Asia, 2018             4621
Euros 2016, Qualifiers                       3934
World Cup Qualifiers, Europe, 2014           3614
Nations League 2020-2021                     3508
Nations League 2022-2023                     3496
World Cup Qualifiers, Africa, 2022           3115
Nations League 2018-2020                     3000
World Cup Qualifiers, Africa, 2018           2498
World Cup 2022, Qualifiers, CONCACAF         2324
World Cup 2022, Qualifiers, CONMEBOL         1820
World Cup 2018, Qualifiers, CONMEBOL         1769
World Cup 2018, Qualifiers, CONCACAF         1435
World Cup 2018                               1393
World Cup 2022                               1384
World Cup 2014                        

In [4]:
dict_of_matches_concacaf_conmebol = {}
big_df_combined['To Remove'] = False

## Big DF

In [35]:


set_of_competitions = ['World Cup 2014, Qualifiers, CONCACAF', 'World Cup 2014, Qualifiers, CONMEBOL', 'World Cup 2018, Qualifiers, CONCACAF',
                       'World Cup 2018, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONCACAF']
num_rows = 0
for index, row in big_df_combined.iterrows():
#for i in range(1772, 1774):
    #index = i
    #row = big_df_combined.loc[i]
    if row['Competition'] in set_of_competitions:
        
        key = f"{row['Match']}:{row['Date']}"
        print(index, key)
        if key in dict_of_matches_concacaf_conmebol.keys():
            print('ye')
        else:
            dict_of_matches_concacaf_conmebol[key] = row['Match ID']
        
        big_df_combined.at[index, 'To Remove'] = True
        num_rows += 1
print(num_rows)

0


In [36]:
big_df_removed = big_df_combined[big_df_combined['To Remove'] == True]

In [8]:
big_df_combined = big_df_combined[big_df_combined['To Remove'] != True]

In [38]:
big_df_combined.to_csv('CONCACAF & CONMEBOL REMOVED - Combined DF.csv')

## Red Group

In [10]:
red_group['To Remove'] = False

In [11]:


set_of_competitions = ['World Cup 2014, Qualifiers, CONCACAF', 'World Cup 2014, Qualifiers, CONMEBOL', 'World Cup 2018, Qualifiers, CONCACAF',
                       'World Cup 2018, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONCACAF']
num_rows = 0
for index, row in red_group.iterrows():
#for i in range(1772, 1774):
    #index = i
    #row = big_df_combined.loc[i]
    if row['Competition'] in set_of_competitions:
        print(index, key)
        key = f"{row['Match']}:{row['Date']}"

        if key in dict_of_matches_concacaf_conmebol.keys():
            print('ye')
        else:
            dict_of_matches_concacaf_conmebol[key] = row['Match ID']
        
        red_group.at[index, 'To Remove'] = True
        num_rows += 1
print(num_rows)

158 Panama vs Canada:30-Mar-22
ye
159 Paraguay vs Venezuela:10-Oct-17
ye
160 Brazil vs Venezuela:13-Nov-20
ye
161 Venezuela vs Chile:17-Nov-20
ye
162 Uruguay vs Paraguay:03-Jun-21
ye
163 Argentina vs Chile:03-Jun-21
ye
164 Colombia vs Argentina:08-Jun-21
ye
165 Paraguay vs Brazil:08-Jun-21
ye
166 Bolivia vs Colombia:02-Sep-21
ye
167 Ecuador vs Paraguay:02-Sep-21
ye
168 Ecuador vs Paraguay:02-Sep-21
ye
169 Venezuela vs Argentina:02-Sep-21
ye
170 Venezuela vs Argentina:02-Sep-21
ye
171 Chile vs Brazil:02-Sep-21
ye
172 Ecuador vs Chile:05-Sep-21
ye
206 Paraguay vs Colombia:05-Sep-21
ye
207 Panama vs Honduras:13-Jun-17
ye
208 Trinidad and Tobago vs Honduras:01-Sep-17
ye
209 Mexico vs Trinidad and Tobago:06-Oct-17
ye
210 Costa Rica vs Honduras:07-Oct-17
ye
211 Panama vs Costa Rica:10-Oct-17
ye
212 Suriname vs Cayman Islands:24-Mar-21
ye
222 Aruba vs Suriname:27-Mar-21
ye
223 Suriname vs Bermuda:04-Jun-21
ye
224 Canada vs Suriname:08-Jun-21
ye
225 Canada vs Honduras:02-Sep-21
ye
226 Canada v

In [12]:
red_group_removed = red_group[red_group['To Remove'] == True]

In [14]:
red_group = red_group[red_group['To Remove'] != True]

In [39]:
red_group.to_csv('CONCACAF & CONMEBOL OUT - Wrong DFs Merged.csv')

## PINK Group 

### Cyrillic

In [17]:
pink_group_cyrillic['To Remove'] = False

In [18]:


set_of_competitions = ['World Cup 2014, Qualifiers, CONCACAF', 'World Cup 2014, Qualifiers, CONMEBOL', 'World Cup 2018, Qualifiers, CONCACAF',
                       'World Cup 2018, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONCACAF']
num_rows = 0
for index, row in pink_group_cyrillic.iterrows():
#for i in range(1772, 1774):
    #index = i
    #row = big_df_combined.loc[i]
    if row['Competition'] in set_of_competitions:
        print(index, key)
        key = f"{row['Match']}:{row['Date']}"

        if key in dict_of_matches_concacaf_conmebol.keys():
            print('ye')
        else:
            dict_of_matches_concacaf_conmebol[key] = row['Match ID']
        
        pink_group_cyrillic.at[index, 'To Remove'] = True
        num_rows += 1
print(num_rows)

0


### Weird

In [19]:
pink_group_odd_name_cases['To Remove'] = False

In [20]:


set_of_competitions = ['World Cup 2014, Qualifiers, CONCACAF', 'World Cup 2014, Qualifiers, CONMEBOL', 'World Cup 2018, Qualifiers, CONCACAF',
                       'World Cup 2018, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONCACAF']
num_rows = 0
for index, row in pink_group_odd_name_cases.iterrows():
#for i in range(1772, 1774):
    #index = i
    #row = big_df_combined.loc[i]
    if row['Competition'] in set_of_competitions:
        print(index, key)
        key = f"{row['Match']}:{row['Date']}"

        if key in dict_of_matches_concacaf_conmebol.keys():
            print('ye')
        else:
            dict_of_matches_concacaf_conmebol[key] = row['Match ID']
        
        pink_group_odd_name_cases.at[index, 'To Remove'] = True
        num_rows += 1
print(num_rows)

345 British Virgin Islands vs Guatemala:03-27-2021
ye
567 Uruguay vs Bolivia:10-Oct-17
ye
2


In [21]:
pink_group_odd_name_cases_removed = pink_group_odd_name_cases[pink_group_odd_name_cases['To Remove'] == True]

In [22]:
pink_group_odd_name_cases = pink_group_odd_name_cases[pink_group_odd_name_cases['To Remove'] == False]

In [34]:
len(pink_group_odd_name_cases)

pink_group_odd_name_cases.to_csv('CONCACAF & CONMEBOL OUT - odd_name_found_0_cases.csv')

### Online Lookup

In [26]:
pink_group_onlone_lookup['To Remove'] = False

In [27]:


set_of_competitions = ['World Cup 2014, Qualifiers, CONCACAF', 'World Cup 2014, Qualifiers, CONMEBOL', 'World Cup 2018, Qualifiers, CONCACAF',
                       'World Cup 2018, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONMEBOL', 'World Cup 2022, Qualifiers, CONCACAF']
num_rows = 0
for index, row in pink_group_onlone_lookup.iterrows():
#for i in range(1772, 1774):
    #index = i
    #row = big_df_combined.loc[i]
    if row['Competition'] in set_of_competitions:
        print(index, key)
        key = f"{row['Match']}:{row['Date']}"

        if key in dict_of_matches_concacaf_conmebol.keys():
            print('ye')
        else:
            dict_of_matches_concacaf_conmebol[key] = row['Match ID']
        
        pink_group_onlone_lookup.at[index, 'To Remove'] = True
        num_rows += 1
print(num_rows)

112 Bolivia vs Chile:05-Sep-17
ye
113 Saint Vincent and the Grenadines vs British Virgin Islands:30-Mar-21
ye
114 Cuba vs British Virgin Islands:02-Jun-21
ye
145 British Virgin Islands vs Curaçao:05-Jun-21
ye
146 Jamaica vs USA:07-Jun-13
ye
147 Honduras vs Jamaica:11-Jun-13
ye
148 Jamaica vs Costa Rica:10-Sep-13
ye
149 Mexico vs Jamaica:06-Feb-13
ye
150 Jamaica vs Panama:22-Mar-13
ye
151 Costa Rica vs Jamaica:26-Mar-13
ye
199 Jamaica vs Mexico:04-Jun-13
ye
11


In [29]:
pink_group_onlone_lookup_removed = pink_group_onlone_lookup[pink_group_onlone_lookup['To Remove'] == True]
len(pink_group_onlone_lookup_removed)

11

In [30]:
pink_group_onlone_lookup = pink_group_onlone_lookup[pink_group_onlone_lookup['To Remove'] != True]

In [33]:
len(pink_group_onlone_lookup)

pink_group_onlone_lookup.to_csv('CONCACAF & CONMEBOL OUT - Online lookup required - 0 names found.csv')

## Saving dict

In [46]:
pink_group_onlone_lookup[pink_group_onlone_lookup['Competition'].isin(set_of_competitions)]

,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,To Remove


In [48]:
dict_of_matches_concacaf_conmebol

{'Jamaica vs Honduras:30-Mar-22': 4198,
 'Colombia vs Bolivia:22-Mar-13': 4240,
 'Uruguay vs Paraguay:22-Mar-13': 4241,
 'Argentina vs Venezuela:22-Mar-13': 4242,
 'Peru vs Chile:22-Mar-13': 4243,
 'Bolivia vs Argentina:26-Mar-13': 4244,
 'Ecuador vs Paraguay:26-Mar-13': 4245,
 'Venezuela vs Colombia:26-Mar-13': 4246,
 'Chile vs Uruguay:26-Mar-13': 4247,
 'Bolivia vs Venezuela:07-Jun-13': 4248,
 'Canada vs Dominica:16-Jun-15': 3997,
 'USA vs Saint Vincent and the Grenadines:13-Nov-15': 4010,
 'Guatemala vs Trinidad and Tobago:13-Nov-15': 4011,
 'Canada vs Honduras:13-Nov-15': 4012,
 'Mexico vs El Salvador:13-Nov-15': 4013,
 'Costa Rica vs Haiti:13-Nov-15': 4014,
 'Jamaica vs Panama:13-Nov-15': 4015,
 'Honduras vs Mexico:17-Nov-15': 4016,
 'Saint Vincent and the Grenadines vs Guatemala:17-Nov-15': 4017,
 'Haiti vs Jamaica:17-Nov-15': 4018,
 'Trinidad and Tobago vs USA:17-Nov-15': 4019,
 'El Salvador vs Canada:17-Nov-15': 4020,
 'Panama vs Costa Rica:17-Nov-15': 4021,
 'Saint Vincent and

In [49]:
import json
with open('matches_IDs_dict.json', 'w') as json_file:
    json.dump(dict_of_matches_concacaf_conmebol, json_file)

In [47]:
dict_of_matches_concacaf_conmebol

800